# Import design libraries

In [89]:
import gdspy
import numpy as np
from importlib import reload
import libraries.general_design_functions as gdf
import libraries.general_sample_creator as creator
import libraries.squid3JJ
import libraries.JJ4q
from copy import deepcopy
# reload(gdspy)
# from design_coaxmons import *

# from loops_and_JJs_coaxmons import*
# from coaxmons import *
# from airbridges import *

In [49]:
reload(gdspy)
reload(creator)
reload(gdf)

<module 'libraries.general_design_functions' from 'E:\\Github\\QCreator\\libraries\\general_design_functions.py'>

# Specify constants, everything in microns

Parameters for the transmission lines

In [50]:
tl_core = 10
tl_gap = 10
tl_ground = 20

In [51]:
watch=lambda: gdspy.LayoutViewer(depth=0,pattern={'default': 8},background='#FFFFFF') #this opens a viewer

In [52]:
reload(gdspy)
reload(creator)
reload(gdf)

<module 'libraries.general_design_functions' from 'E:\\Github\\QCreator\\libraries\\general_design_functions.py'>

# Sample generation

## borders with pads

Function names should be lowercase, with words separated by underscores as necessary to improve readability.

Variable names follow the same convention as function name

In [170]:
reload(gdspy)
reload(creator)
reload(gdf)
layers_configuration={
    'total':0,
    'restricted area':10,
    'for removing':100,
    'JJs':1,
    'air bridges':2,
    'air bridge pads':3,
    'vertical gridlines':4,
    'horizontal gridlines':5
}
sample = creator.Sample('4_qubits',layers_configuration)

#specify sample vertical and horizontal lengths
sample.sample_vertical_size=4.7e3
sample.sample_horizontal_size=4.7e3
# be aware of the fact that the minimum distance beetween structures must be bigger than 4um
# min_distance = 4

#specify pads for the sample
sample.add_pad(tl_core,tl_gap,tl_ground,(760,2360))
sample.add_pad(tl_core,tl_gap,tl_ground,(3930, 2360))
sample.add_pad(tl_core,tl_gap,tl_ground,(1111,3905-10))
sample.add_pad(tl_core,tl_gap,tl_ground,(1111+1250,3905-10))
sample.add_pad(tl_core,tl_gap,tl_ground,(1111+1250*2,3905-10))
sample.add_pad(tl_core,tl_gap,tl_ground,(1111,770))
sample.add_pad(tl_core,tl_gap,tl_ground,(1111+1250,770))
sample.add_pad(tl_core,tl_gap,tl_ground,(1111+1250*2,770))

#generate borders with pads
sample.generate_sample_edges_and_pads()

## first coaxmon qubit

### coaxmon qubit

In [171]:
shift=-1/10
phi1=1/5
couplers=[gdf.QubitCoupler(arc_start=-1/6-1/100+shift,arc_finish=-3/6+1/100+shift,phi=-1/2, w =4),
          gdf.QubitCoupler(arc_start=3/6+1/100+shift,arc_finish=5/6-1/100+shift,phi=phi1, w =4),
          gdf.QubitCoupler(arc_start=1/6+1/100+shift,arc_finish=3/6-1/100+shift,phi=1/2, w =4),
          gdf.QubitCoupler(arc_start=-1/6+1/100+1+shift,arc_finish=1/6-1/100+1+shift,phi=1, w =10),
          gdf.QubitCoupler(arc_start=-1/6+1/100+shift,arc_finish=1/6-1/100+shift,phi=0, w =10),
          gdf.QubitCoupler(arc_start=-5/6+1/100+shift,arc_finish=-3/6-1/100+shift,phi=1, w =10, grounded=True)]
JJ_coaxmon1 = {'a1':15,
               'b1':0.8,
               'a2':0.45,
               'b2':0.243,
               'c1':0.243,
               'c2':10,
               'angle_qubit':3*np.pi/2-1*np.pi/3,
               'angle_JJ':0}

sample.add_qubit(gdf.Coaxmon,*[100,140,200,230,250,couplers,JJ_coaxmon1],**{'angle':0,'coordinate':(1750,1750)})

### fluxline

In [172]:
points = [sample.pads[5].coordinate,(1111,1230),(1500,1350)]
bridge_params=creator.Bridges_over_line_param(distance=90, offset=40,
                    width=15, length=90, padsize=(30,30), line_type=None)
sample.generate_line(points, tl_core, tl_gap, tl_ground, None,bridge_params=bridge_params)
                     
fluxline=[4,4,4]
point = sample.generate_narrowing_part(sample.lines[-1],fluxline)

end = {'type':'fluxline',
       'JJ': sample.qubits[0].JJ,
        'length':50,
        "width":4}
points = [point,
         (point[0]+10*np.cos(sample.lines[-1].angle),point[1]+10*np.sin(sample.lines[-1].angle)),
         (1663,1450),
          (sample.qubits[0].JJ._x0,sample.qubits[0].JJ._y0-45-50)]
sample.generate_line(points,4,3,4,None,end)
# sample.generate_bridge((sample.lines[-1].end[0],sample.lines[-1].end[1]),15, 90, 30,sample.lines[-1].angle+np.pi/2)


## second coaxmon qubit

In [173]:
JJ_coaxmon1 = {'a1':10,
               'b1':0.8,
               'a2':0.45,
               'b2':0.243,
               'c1':0.243,
               'c2':10,
               'angle_qubit':-np.pi/4,
               'angle_JJ':0}
sample.add_qubit(gdf.Coaxmon,*[100,140,200,230,250,couplers,JJ_coaxmon1],**{'angle':1*np.pi/2,'coordinate':(1750+1250,1750)})


In [174]:
sample.finish_him()
watch()

<gdspy.viewer.LayoutViewer object .!layoutviewer>

### fluxline

In [175]:
points = [(1111+1250*2,770),(1111+1250*2,1230),(3206,1311)]
bridge_params=creator.Bridges_over_line_param(distance=90, offset=40,
                    width=15, length=90, padsize=(30,30), line_type=None)
sample.generate_line(deepcopy(points), tl_core, tl_gap, tl_ground, None,bridge_params=bridge_params)
fluxline=[4,3,4]
# point = sample.lines[-1].end
point = sample.generate_narrowing_part(sample.lines[-1],fluxline)

end = {'type':'fluxline',
       'JJ': sample.qubits[-1].JJ,
        'length':20,
        "width":4}
points = [point,(point[0]+50*np.cos(sample.lines[-1].angle),point[1]+50*np.sin(sample.lines[-1].angle)),
         (3073,1610)]

sample.generate_line(points,4,3,4,None,end)

## third coaxmon qubit

In [176]:
#3 coaxmon
JJ_coaxmon1 = {'a1':10,
               'b1':0.8,
               'a2':0.45,
               'b2':0.243,
               'c1':0.243,
               'c2':10,
               'angle_qubit':+np.pi/4,
               'angle_JJ':np.pi}

sample.add_qubit(gdf.Coaxmon,*[100,140,200,230,250,couplers,JJ_coaxmon1],**{'angle':np.pi,'coordinate':(1750+1250,1750+1250)})

## fourth coaxmon qubit

In [177]:
#4 coaxmon
JJ_coaxmon1 = {'a1':10,
               'b1':0.8,
               'a2':0.45,
               'b2':0.243,
               'c1':0.243,
               'c2':10,
               'angle_qubit':-np.pi/4+np.pi,
               'angle_JJ':np.pi}
sample.add_qubit(gdf.Coaxmon,*[100,140,200,230,250,couplers,JJ_coaxmon1],**{'angle':-np.pi/2,'coordinate':(1750,1750+1250)})

## Qubit couplers

### Qubit 1-2 coupler

In [179]:
coupler_core=10
coupler_gap=25
coupler_ground=20
JJ={'indent':60,
   'angle_JJ':0,
   'a1':0.5,
   'a2':0.5}
squid={'x':2360,
       'y':1750+1,
       'a1':0.5,
       'b1':0.5,
       'a2':0.5,
       'b2':0.5,
       'c1':10,
       'c2':10,
      "angle":0}
sample.add_qubit_coupler(coupler_core,coupler_gap,coupler_ground,sample.qubits[0],sample.qubits[1],JJ,squid)

2040.0 1738.5
2710.0 1738.5


In [182]:
points = [(2360,770),(2360,1500)]
sample.generate_line(points,tl_core,tl_gap,tl_ground,None)
fluxline=[4,3,4]
point = sample.generate_narrowing_part(sample.lines[-1],fluxline)
end = {'type':'fluxline',
       'JJ': sample.couplers[0].squid,
        'length':20,
        "width":4}
points = [point,(point[0]+10*np.cos(sample.lines[-1].angle),point[1]+10*np.sin(sample.lines[-1].angle)),
         (2360,1650)]
sample.generate_line(points,4,3,4,None,end)
sample.generate_bridge((sample.lines[-1].end[0],sample.lines[-1].end[1]-25),15, 90, (30,30),0)
# print(sample.lines[-1].angle)

 Rectangle((2315.0, 1617.5), (2405.0, 1632.5), 2, 0)]

### Qubit 2-3 coupler

In [185]:
JJ={'indent':60,
   'angle_JJ':0,
   'a1':0.5,
   'a2':0.5}
squid={'x':3036,
       'y':2156,
       'a1':0.5,
       'b1':0.5,
       'a2':0.5,
       'b2':0.5,
       'c1':10,
       'c2':10,
      "angle":0}
sample.add_qubit_coupler(coupler_core,coupler_gap,coupler_ground,sample.qubits[1],sample.qubits[2],JJ,squid)

3000.0 2011.0
3000.0 2681.0


In [186]:
points = [(3930, 2360),(3800,2360),(3209,2090)]
sample.generate_line(points,tl_core,tl_gap,tl_ground,None)
fluxline=[4,3,4]
point = sample.generate_narrowing_part(sample.lines[-1],fluxline)
end = {'type':'fluxline',
       'JJ': sample.couplers[1].squid,
        'length':20,
        "width":4}
points = [point,(point[0]+10*np.cos(sample.lines[-1].angle),point[1]+10*np.sin(sample.lines[-1].angle)),
         (3060,2090)]
sample.generate_line(points,4,3,4,None,end)
# sample.generate_bridge((sample.lines[-1].end[0],sample.lines[-1].end[1]-25),15, 90, 30,3*np.pi/2)

In [187]:
sample.finish_him()
watch()

<gdspy.viewer.LayoutViewer object .!layoutviewer>

### Qubit 3-4 coupler

In [45]:
JJ={'indent':60,
   'angle_JJ':0,
   'a1':0.5,
   'a2':0.5}
squid={'x':2360,
       'y':3000,
       'a1':0.5,
       'b1':0.5,
       'a2':0.5,
       'b2':0.5,
       'c1':10,
       'c2':10,
      "angle":np.pi}
sample.add_qubit_coupler(coupler_core,coupler_gap,coupler_ground,sample.coaxmons[2],sample.coaxmons[3],JJ,squid)

2710.0 2988.5
2040.0 2988.5


In [46]:
#third coupler  fluxline
points = [sample.pads[3].coordinate,(2360,3200)]
sample.generate_line(points,tl_core,tl_gap,tl_ground,None)
fluxline=[4,3,4]
point = sample.generate_narrowing_part(sample.lines[-1],fluxline)
end = {'type':'fluxline',
       'JJ': sample.couplers[2].squid,
        'length':20,
        "width":4}
points = [point,(point[0]+10*np.cos(sample.lines[-1].angle),point[1]+10*np.sin(sample.lines[-1].angle)),
         (2360,3016)]
sample.generate_line(points,4,3,4,None,end)
# sample.generate_bridge((sample.lines[-1].end[0],sample.lines[-1].end[1]-25),15, 90, 30,3*np.pi/2)

In [21]:
# sample.create_grid(100,100)
sample.finish_him()
watch()

<gdspy.viewer.LayoutViewer object .!layoutviewer>

## Feedline

In [21]:
points = [(760,2360),(2100,2360)]
sample.generate_line(points,tl_core,tl_gap,tl_ground,None)
# couplers = sample.generate_resonator_coupler(sample.lines[-1].end,(2320,2360),TL_core,TL_gap,TL_ground,4,4,4,2.5,0,'1')

### Resonator+Purcell

#### Purcell

In [9]:
#purcell
# start_x=-20
# start_y=40
# ap=130
# bp=510
# purcell_point=(2340.75,2340.5)
# par_end, par_begin = sample.generate_purcell(purcell_begin = (purcell_point[0]-ap-start_x,purcell_point[1]-bp+150-start_y),
#                         purcell_end=(purcell_point[0]-start_x,purcell_point[1]-start_y), 
#                         restricted_area_points0=(purcell_point[0]-ap-start_x,purcell_point[1]-bp-start_y),
#                         a_restricted_area=ap, b_restricted_area=bp,  
#                         purcell_core=4, purcell_ground=4, purcell_gap=4, length_of_purcell_meander=3000,
#              begin_coupler=None)

# points = [par_begin,(par_begin[0]+5,par_begin[1]),
#          (par_begin[0]+5,par_begin[1]+17),(couplers[1][0],par_begin[1]+17),couplers[1]]
# sample.generate_line(points,4,4,4,None,corner_type=None)


# points = [couplers[0],(couplers[0][0],couplers[0][1]-200),(couplers[0][0]+20,couplers[0][1]-200),
#          (couplers[0][0]+20,couplers[0][1]-10),(couplers[0][0]+150,couplers[0][1]-10)]
# end_type={'type':'open',
#           'gap':10,
#           'ground':10}
# sample.generate_line(points,4,4,4,None,end=end_type,corner_type=None)





# couplers = sample.generate_resonator_coupler((2100,2360),(2320,2360),TL_core,TL_gap,TL_ground,4,4,4,2.5,0,'1')


In [20]:
watch()

<gdspy.viewer.LayoutViewer object .!layoutviewer>

In [11]:
# sample.create_grid(100,100)
sample.finish_him()
watch()

<gdspy.viewer.LayoutViewer object .!layoutviewer>

In [11]:
# feedline
points = [(760,2360),(1920,2360),(2010,2560),(2100,2400)]
sample.generate_line(points,TL_core,TL_gap,TL_ground,None)


# #resonator
# ar=180
# br=400
# coupler_length=70
# resonator_point=(2340.75,2340.5)
# res_begin, res_end =sample.generate_resonator(resonator_begin=(resonator_point[0]-ap-coupler_length-ar-start_x,
#                                                                resonator_point[1]-br+250-start_y-200), 
#                           resonator_end=(resonator_point[0]-ap-coupler_length-start_x,
#                                          resonator_point[1]-bp-start_y+150), 
#                           restricted_area_points0=(resonator_point[0]-ap-coupler_length-ar-start_x,
#                                                    resonator_point[1]-br-start_y), 
#                           a_restricted_area=ar, 
#                           b_restricted_area=br,
#                           resonator_core=4, resonator_ground=4, resonator_gap=4, length_of_all_resonator=3800,
#                angle=0)


# sample.generate_coupler_purcell_resonator(res_end,par_end,
#                                          [4,4,4],
#                                          [4,4,4],
#                                          5,20,20+10)# тут косяк, надо починить

# distance=(res_begin[0]-
#          (sample.coaxmons[0].center.x+(sample.coaxmons[0].R4)*np.cos(np.pi*phi1)))/np.cos(np.pi*phi1)
# points = [(res_begin[0],res_begin[1]),
#           (res_begin[0],res_begin[1]-20),
#           (sample.coaxmons[0].center.x+(sample.coaxmons[0].R4+distance)*np.cos(np.pi*phi1),
#          sample.coaxmons[0].center.y+(sample.coaxmons[0].R4+distance)*np.sin(np.pi*phi1)),
#          (sample.coaxmons[0].center.x+(sample.coaxmons[0].R4-0.1)*np.cos(np.pi*phi1),
#          sample.coaxmons[0].center.y+(sample.coaxmons[0].R4-0.1)*np.sin(np.pi*phi1))]
# sample.generate_line(points,4,4,4,None,corner_type='rect')


# #second resonator and purcell 
# points = [(2320,2360),(2400,2360)]
# sample.generate_line(points,TL_core,TL_gap,TL_ground,None)

# start_x=-20
# start_y=40
# ap=130
# bp=510
# # coupler_end=(2820,2360)
# couplers = sample.generate_resonator_coupler(sample.lines[-1].end,(2820,2360),TL_core,TL_gap,TL_ground,4,4,4,2.5,0,'1')
# par_begin,par_end = sample.generate_purcell(purcell_begin = (2440.75-ap-start_x,2340.5-bp+150-start_y),
#                         purcell_end=(2440.75-start_x,2340.5-start_y), 
#                         restricted_area_points0=(2440.75-ap-start_x,2340.5-bp-start_y),
#                         a_restricted_area=ap, b_restricted_area=bp,  
#                         purcell_core=4, purcell_ground=4, purcell_gap=4, length_of_purcell_meander=3000,
#              begin_coupler=None,type_mirror="mirror")

# points = [par_begin,(par_begin[0]-5,par_begin[1]),
#          (par_begin[0]-5,par_begin[1]+17),(couplers[1][0],par_begin[1]+17),couplers[1]]
# sample.generate_line(points,4,4,4,None,corner_type=None)

# points = [couplers[0],(couplers[0][0],couplers[0][1]-500)]
# end_type={'type':'open',
#           'gap':10,
#           'ground':10}
# sample.generate_line(points,4,4,4,None,end=end_type,corner_type=None)

# ar=240
# br=325
# coupler_length=70
# res_begin, res_end =sample.generate_resonator(resonator_begin=(2850.75-ap-coupler_length-ar-start_x,2340.5-br+250-start_y-200), 
#                           resonator_end=(2850.75-ap-coupler_length-start_x,2340.5-bp+150-start_y+100), 
#                           restricted_area_points0=(2850.75-ap-coupler_length-ar-start_x,2340.5-br-start_y), 
#                           a_restricted_area=ar, 
#                           b_restricted_area=br,
#                           resonator_core=4, resonator_ground=4, resonator_gap=4, length_of_all_resonator=3800,
#                angle=0,type_mirror="mirror")


# points = [res_begin,(res_begin[0],par_end[1])]

# sample.generate_line(points,4,4,4,None)

# sample.generate_coupler_purcell_resonator(par_end,
#                                           points[-1],
#                                          [4,4,4],
#                                          [4,4,4],
#                                          10,20,20+10)# тут косяк, надо починить

# points = [res_begin,(res_begin[0],par_end[1])]
# sample.generate_line(points,4,4,4,None)

# points = [coupler_end,(coupler_end[0]+105,coupler_end[1])]
# sample.generate_line(points,TL_core,TL_gap,TL_ground,None)
# end_point, center_point = sample.generate_bridge_over_feedline(sample.lines[-1], 
#                                              [15, 90, 30],
#                                              [TL_core,TL_gap,TL_ground],
#                                              5)
# points = [end_point,(3400,2360)]
# end = {'type':'open',
#        'ground':10,
#        'gap':10}
# sample.generate_line(points,TL_core,TL_gap,TL_ground,None,end)




<gdspy.viewer.LayoutViewer object .!layoutviewer>

In [17]:
sample.finish_him()
# 
sample.create_grid(100,100)

gdspy.LayoutViewer(depth=0,pattern={'default': 8},background='#FFFFFF') #this opens a viewer

AttributeError: 'Sample' object has no attribute 'sample_size_x'

In [46]:
def length(freq):
    c=299792458
    n=np.sqrt((11.45+1)/2)
    return c/(4*n*freq*1e9)*1e3

In [48]:
length(7),length(9)

(4.291340681252264, 3.3377094187517606)

In [37]:
calculate_total_length(points)

1062.6718739192922

In [12]:
gdspy.write_gds('test1.gds', cells=None, name='library', unit=1e-06, precision=1e-09, timestamp=None, binary_cells=None)